In [3]:
path_to_file = 'SW_EpisodeIV.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
type(text)

str

In [5]:
text[:10]

'"character'

In [ ]:
text_lines = text.split('\n')[1:-1]
text_lines

In [7]:
text_lines[-5:]

['"1006" "LUKE" "Oh, no!"',
 '"1007" "THREEPIO" "Oh, my!  Artoo!  Can you hear me?  Say something!You can repair him, can\'t you?"',
 '"1008" "TECHNICIAN" "We\'ll get to work on him right away."',
 '"1009" "THREEPIO" "You must repair him!  Sir, if any of my circuits or gears will help, I\'ll gladly donate them."',
 '"1010" "LUKE" "He\'ll be all right."']

In [8]:
real_text = []
for x in text_lines:
    real_text.append(x.split('"')[5])
real_text

["Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness!",
 "We're doomed!",
 "There'll be no escape for the Princess this time.",
 "What's that?",
 'I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...',
 'Hurry up!  Come with me!  What are you waiting for?!  Get in gear!',
 'Artoo! Artoo-Detoo, where are you?',
 'At last!  Where have you been?',
 "They're heading in this direction. What are we going to do?  We'll be sent to the spice mines of Kessel or smashed into who knows what!",
 'Wait a minute, where are you going?',
 'The Death Star plans are not in the main computer.',
 'Where are those transmissions you intercepted?',
 'We intercepted no transmissions. Aaah...  This is a consular ship. Were on a diplomatic mission.',
 'If this is a consular ship... where is the Ambassador?',
 "Commander, tear this ship apart until you've found those plans and bring me the Ambassador.  I want h

In [9]:
text = x = ' '.join(real_text)
text

"Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness! We're doomed! There'll be no escape for the Princess this time. What's that? I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister... Hurry up!  Come with me!  What are you waiting for?!  Get in gear! Artoo! Artoo-Detoo, where are you? At last!  Where have you been? They're heading in this direction. What are we going to do?  We'll be sent to the spice mines of Kessel or smashed into who knows what! Wait a minute, where are you going? The Death Star plans are not in the main computer. Where are those transmissions you intercepted? We intercepted no transmissions. Aaah...  This is a consular ship. Were on a diplomatic mission. If this is a consular ship... where is the Ambassador? Commander, tear this ship apart until you've found those plans and bring me the Ambassador.  I want her alive! There she is! Set for stun! She'll be all right

In [10]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

63 unique characters


In [11]:
vocab

[' ',
 '!',
 "'",
 ',',
 '-',
 '.',
 '2',
 '3',
 '8',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 '\\',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

## Process the text - Vectorization

In [12]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(61648,), dtype=int64, numpy=array([15, 46, 41, ..., 45, 57,  6], dtype=int64)>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

D
i
d
 
y
o
u
 
h
e


In [30]:
seq_length = 150
examples_per_epoch = len(text)//(seq_length+1)

In [31]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'D' b'i' b'd' b' ' b'y' b'o' b'u' b' ' b'h' b'e' b'a' b'r' b' ' b't'
 b'h' b'a' b't' b'?' b' ' b' ' b'T' b'h' b'e' b'y' b"'" b'v' b'e' b' '
 b's' b'h' b'u' b't' b' ' b'd' b'o' b'w' b'n' b' ' b't' b'h' b'e' b' '
 b'm' b'a' b'i' b'n' b' ' b'r' b'e' b'a' b'c' b't' b'o' b'r' b'.' b' '
 b' ' b'W' b'e' b"'" b'l' b'l' b' ' b'b' b'e' b' ' b'd' b'e' b's' b't'
 b'r' b'o' b'y' b'e' b'd' b' ' b'f' b'o' b'r' b' ' b's' b'u' b'r' b'e'
 b'.' b' ' b' ' b'T' b'h' b'i' b's' b' ' b'i' b's' b' ' b'm' b'a' b'd'
 b'n' b'e' b's' b's' b'!' b' ' b'W' b'e' b"'" b'r' b'e' b' ' b'd' b'o'
 b'o' b'm' b'e' b'd' b'!' b' ' b'T' b'h' b'e' b'r' b'e' b"'" b'l' b'l'
 b' ' b'b' b'e' b' ' b'n' b'o' b' ' b'e' b's' b'c' b'a' b'p' b'e' b' '
 b'f' b'o' b'r' b' ' b't' b'h' b'e' b' ' b'P' b'r' b'i'], shape=(151,), dtype=string)


In [32]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness! We're doomed! There'll be no escape for the Pri"
b"ncess this time. What's that? I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister... Hurry up!  Come "
b"with me!  What are you waiting for?!  Get in gear! Artoo! Artoo-Detoo, where are you? At last!  Where have you been? They're heading in this direction."
b" What are we going to do?  We'll be sent to the spice mines of Kessel or smashed into who knows what! Wait a minute, where are you going? The Death Sta"
b'r plans are not in the main computer. Where are those transmissions you intercepted? We intercepted no transmissions. Aaah...  This is a consular ship.'


In [33]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [34]:
dataset = sequences.map(split_input_target)

In [35]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness! We're doomed! There'll be no escape for the Pr"
Target: b"id you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness! We're doomed! There'll be no escape for the Pri"


## Batching dataset

In [36]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 150), dtype=tf.int64, name=None), TensorSpec(shape=(64, 150), dtype=tf.int64, name=None))>

In [37]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [38]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [39]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [40]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 64) # (batch_size, sequence_length, vocab_size)


In [41]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

In [42]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
history = model.fit(dataset,
                    epochs=100,
                    callbacks=[checkpoint_callback])

Epoch 1/100
6/6 [==============================] - 1s 166ms/step - loss: 2.1742
Epoch 2/100
6/6 [==============================] - 1s 163ms/step - loss: 2.1563
Epoch 3/100
6/6 [==============================] - 1s 166ms/step - loss: 2.1322
Epoch 4/100
6/6 [==============================] - 1s 162ms/step - loss: 2.1083
Epoch 5/100
6/6 [==============================] - 1s 165ms/step - loss: 2.0887
Epoch 6/100
6/6 [==============================] - 1s 165ms/step - loss: 2.0650
Epoch 7/100
6/6 [==============================] - 1s 177ms/step - loss: 2.0441
Epoch 8/100
6/6 [==============================] - 1s 165ms/step - loss: 2.0182
Epoch 9/100
6/6 [==============================] - 1s 164ms/step - loss: 1.9991
Epoch 10/100
6/6 [==============================] - 1s 165ms/step - loss: 1.9743
Epoch 11/100
6/6 [==============================] - 1s 165ms/step - loss: 1.9521
Epoch 12/100
6/6 [==============================] - 1s 164ms/step - loss: 1.9271
Epoch 13/100
6/6 [===================

In [48]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [49]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [54]:
import time

start = time.time()
states = None
next_char = tf.constant([' '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

 I nead you ase. That's impossible!  How will the Emperor my all light.  She may net be of your season... They're togathe droids for some tomed. Don't werreg about, the blast doors.  If oth oken any other way out. I can't hole them off forever!  Now what? This is some rescue. But it's any worry about the were thousand sometsis can figu.  It's looking formale something commate no meached hos miselutten. Aly don't cap here.  It him no, kid!  But they're coming in too fast! Bore on the door, Artoo. All right, check that side word if it's tares of this stat if you randed contincess, tee onttoing to big fur you kid.  I forgot I tulf your hid. Now droids drench a bad this station. Red Five, where are you? I can't seaving and wait for my sign.  That's impossible! Come on, moreys! Okey have my peace and Threeding es. No soce! I tooded ar They's hears.  Ase the o beldry alerat.  If you never help!  Cle ine that meffact short into wnow, I'm such de? Pirnibl ship chatch on unselt shields.  You'll